# CS155 Miniproject 1

zchen@caltech.edu

Sentiment Analysis via Neural Networks

In [3]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import sklearn.model_selection

import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LeakyReLU

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Loading Dataset

In [4]:
# Load final validation data set
FV_data = np.loadtxt('../data/test_data.txt',delimiter=' ',skiprows=1)
# Load input training data set
train_data = np.loadtxt('../data/training_data.txt',delimiter=' ',skiprows=1)

# Get header words list
f = open('../data/test_data.txt','r')
words = np.array(f.readline().split())
f.close()

# Splot y_train and x_train from training set
x_tall = train_data[:,1:]
y_tall = train_data[:,0]

# One hot encode categories
y_tall = keras.utils.np_utils.to_categorical(y_tall)

## Trying Neural Networks on bag of words dataset

In [5]:
# Function to generate DNN of given depth and width
def getModel(layers,Pdrop):
    model = Sequential()
    model.add(Dense(layers[0],input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(Pdrop))
    for i in layers[1:]:
        model.add(Dense(i))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    
    # predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))
    return model

In [58]:
# Compile the DNN model
def trainModels(k,x_tall,y_tall):
    # Storage for k fold cross validation    
    trainErr = []
    testErr = []
    acc = []
    
    # Split the training data k-fold number of ways for k-fold validation of the learning algorithm
    kf = sklearn.model_selection.KFold(n_splits=k)
    inds = [ind for ind in kf.split(x_tall, y_tall)]
    
    i=0
    for train,val in inds:
        # Training and validation data for k fold cross validation
        Xtrain = x_tall[train]
        Ytrain = y_tall[train]
        Xval = x_tall[val]
        Yval = y_tall[val]
    
        # Define the DNN model
        model = getModel([500,250,125],0.4)
        
        # Compile it and fit
        model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
        model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=10,verbose=1,validation_data=(Xval, Yval))
        
        # store training and test error
        trainErr.append(model.evaluate(x=Xtrain, y=Ytrain))
        testErr.append(model.evaluate(x=Xval, y=Yval))
        acc.append(getAccuracy(model,Xval,Yval))

        # Status output
        print('k-iteration = ',i)
        i=i+1
        
    return np.array(trainErr),np.array(testErr),np.array(acc)

# undo one hot encoding
def Unencode(out):
    ypred = out[:,0] < out[:,1]
    return ypred

# Function to get explicit model accuracy from softmax
def getAccuracy(model,xt,yt):
    out = model.predict(xt)
    ypred = Unencode(out)
    ytrue = Unencode(yt)
    acc = 1.0*np.sum(ypred == ytrue)/len(ytrue)
    return acc

In [59]:
trainErr,testErr,Vacc = trainModels(5,x_tall,y_tall)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 6s 399us/step - loss: 0.4819 - acc: 0.7697 - val_loss: 0.3581 - val_acc: 0.8390
Epoch 2/10
16000/16000 [==============================] - 2s 146us/step - loss: 0.3154 - acc: 0.8621 - val_loss: 0.3339 - val_acc: 0.8655
Epoch 3/10
16000/16000 [==============================] - 2s 146us/step - loss: 0.2547 - acc: 0.8930 - val_loss: 0.3446 - val_acc: 0.8582
Epoch 4/10
16000/16000 [==============================] - 2s 149us/step - loss: 0.2089 - acc: 0.9137 - val_loss: 0.3923 - val_acc: 0.8548
Epoch 5/10
16000/16000 [==============================] - 2s 147us/step - loss: 0.1729 - acc: 0.9291 - val_loss: 0.4131 - val_acc: 0.8560
Epoch 6/10
16000/16000 [==============================] - 2s 147us/step - loss: 0.1319 - acc: 0.9495 - val_loss: 0.4641 - val_acc: 0.8505
Epoch 7/10
16000/16000 [==============================] - 2s 147us/step - loss: 0.1084 - acc: 0.9585 - val_loss: 0.4971 - v

In [63]:
print('\n trainErr')
print(trainErr)
print('avg trainErr',np.mean(trainErr,axis=0))
print('\n testErr')
print(testErr)
print('avg testErr',np.mean(testErr,axis=0))
print('\n Accuracy')
print(Vacc)
print('avg acc',np.mean(Vacc))



 trainErr
[[0.00951698 0.9980625 ]
 [0.00941472 0.9986875 ]
 [0.01647534 0.99575   ]
 [0.00923579 0.9983125 ]
 [0.01178948 0.9970625 ]]
avg trainErr [0.01128646 0.997575  ]

 testErr
[[0.59455196 0.847     ]
 [0.58926733 0.8465    ]
 [0.67467756 0.8355    ]
 [0.69684073 0.84025   ]
 [0.67334788 0.83125   ]]
avg testErr [0.64573709 0.8401    ]

 Accuracy
[0.847   0.8465  0.8355  0.84025 0.83125]
avg acc 0.8401


Hyperparameters plots

Validation accuracy plateaus at 85%

Debugging training and test errors

In [53]:
# Split the training data k-fold number of ways for k-fold validation of the learning algorithm
k=5
kf = sklearn.model_selection.KFold(n_splits=k)
inds = [ind for ind in kf.split(x_tall, y_tall)]

i=0
train,val = inds[0]
# Training and validation data for k fold cross validation
Xtrain = x_tall[train]
Ytrain = y_tall[train]
Xval = x_tall[val]
Yval = y_tall[val]

# Define the DNN model
model = getModel([500,250,125],0.4)

# Compile it and fit
model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=10,verbose=1,validation_data=(Xval, Yval))
ypred = model.predict(Xval,batch_size=2**8,verbose=1)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 6s 365us/step - loss: 0.4876 - acc: 0.7648 - val_loss: 0.3556 - val_acc: 0.8410
Epoch 2/10
16000/16000 [==============================] - 2s 145us/step - loss: 0.3199 - acc: 0.8604 - val_loss: 0.3461 - val_acc: 0.8520
Epoch 3/10
16000/16000 [==============================] - 2s 145us/step - loss: 0.2581 - acc: 0.8937 - val_loss: 0.3508 - val_acc: 0.8563
Epoch 4/10
16000/16000 [==============================] - 2s 145us/step - loss: 0.2101 - acc: 0.9151 - val_loss: 0.3738 - val_acc: 0.8520
Epoch 5/10
16000/16000 [==============================] - 2s 146us/step - loss: 0.1709 - acc: 0.9314 - val_loss: 0.4012 - val_acc: 0.8490
Epoch 6/10
16000/16000 [==============================] - 2s 145us/step - loss: 0.1381 - acc: 0.9449 - val_loss: 0.4880 - val_acc: 0.8360
Epoch 7/10
16000/16000 [==============================] - 2s 145us/step - loss: 0.1076 - acc: 0.9592 - val_loss: 0.5194 - v

In [60]:
# Function to get bag of words which were misclassified
def getBagOfWords(xtrain,ypred,ytrue,words):
    out = []
    ypredu = Unencode(ypred).astype(int)
    ytrueu = Unencode(ytrue).astype(int)
    # Get locations of bag of words which were misclassified
    idx = np.arange(0,len(ypredu))
    idxErr = idx[ypredu!=ytrueu]
    Xerr = xtrain[ypredu!=ytrueu]
    j = 0
    for i in Xerr:
        out.append([ytrue[idxErr[j]],ypred[idxErr[j]],words[i>0],i[i>0]])
        j=j+1
    return out

out = getBagOfWords(Xval,ypred,Yval,words)

In [61]:
print(out[0])

[array([0., 1.]), array([9.993092e-01, 6.907701e-04], dtype=float32), array(['thi', 'veri', 'get', 'onli', 'look', 'want', 'see', 'mani', 'new',
       'peopl', 'still', 'need', 'two', 'feel', 'start', 'long', 'listen',
       'excel', 'enough', 'person', 'cover', 'almost', 'scene', 'instead',
       'famili', 'sever', 'hour', 'els', 'fine', 'talk', 'american',
       'entir', 'lack', 'impress', 'state', 'avail', 'certainli',
       'student', 'danc', 'parent', 'critic', 'centuri', 'train',
       'aspect'], dtype='<U12'), array([3., 2., 1., 1., 2., 1., 2., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 2., 2., 6., 1., 1., 1., 1., 1.])]


In [62]:
# Function to write final predictions
def writeResults(out):
    f = open('DNN_submission.txt','w')
    f.write('Id,Prediction\n')
    ypred = Unencode(out)
    for i in range(0,len(ypred)):
        f.write(str(i+1)+','+str(int(ypred[i]))+'\n')
    f.close()

# Compile it and fit
model = getModel([500,250,125,75,25],0.4)
model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
fit = model.fit(x_tall, y_tall, batch_size=2**8, epochs=4,verbose=1)
out = model.predict(FV_data,batch_size=2**8,verbose=1)

writeResults(out)

Epoch 1/4
20000/20000 [==============================] - 7s 368us/step - loss: 0.4787 - acc: 0.7632
Epoch 2/4
20000/20000 [==============================] - 3s 144us/step - loss: 0.3342 - acc: 0.8569
Epoch 3/4
20000/20000 [==============================] - 3s 145us/step - loss: 0.2775 - acc: 0.8807
Epoch 4/4
10000/10000 [==============================] - 2s 213us/step
